<a href="https://colab.research.google.com/github/CaptainJimbo/MyPortfolio/blob/main/myGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Based on "Attention is all you need" paper** [link](https://arxiv.org/abs/1706.03762). This simple algorithm is a Transformer-based Language Model to showcase how an LLM like ChatGPT is trained. It doens't include the pretuning and supervised finetuning.

In [ ]:
import torch

In [ ]:
# I need a "toy" dataset to train with.
# (This is very small comparing to a big chunk of the internet that ChatGPT is trained on!)
# This is a .txt file with some of Shakespeare's works.
# The goal is to create a model that produces Shakespearean language!!
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O input.txt

--2023-07-10 12:07:36--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-07-10 12:07:36 (36.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(f'length of the dataset is {len(text)}')
print(f'\nand here is a random part of the dataset {text[60:464]}')

length of the dataset is 1115394

and here is a random part of the dataset 

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.




In [ ]:
# The algorithm needs to understands characters. But it doesn't need the the particular characters.
# It could be numbers i.e. indices. So I create a mapping from characters to indices.
vocabulary = sorted(list(set(text)))
print('here is the vocabulary, i.e. every possible character that exists in this text.',''.join(vocabulary))

here is the vocabulary, i.e. every possible character that exists in this text. 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [ ]:
char_to_idx = {character:index for index, character in enumerate(vocabulary)}
idx_to_char = {index:character for index, character in enumerate(vocabulary)}

# Mappings
def encode(text):
  return [char_to_idx[character] for character in text]
def decode(indices):
  return ''.join(idx_to_char[index] for index in indices)

#encode('Hello There'), decode(encode('Hello There'))

In [ ]:
data = torch.tensor(encode(text),dtype=torch.long) # This is tensor with indices representing characters.
print('tensor shape',data.shape)
print('tensor  type',data.dtype)
print('tensor  rank',data.dim())

tensor shape torch.Size([1115394])
tensor  type torch.int64
tensor  rank 1


In [ ]:
0.8*100344

80275.20000000001

In [ ]:
# We need a train set and test set
train_data = data[:int(0.8*len(data))]
test_data = data[int(0.8*len(data)):]

In [ ]:
gram_len = 5
X = train_data[:gram_len]
y = train_data[1:gram_len+1]
X, y

(tensor([18, 47, 56, 57, 58]), tensor([47, 56, 57, 58,  1]))

In [ ]:
torch.manual_seed(1337)
batch_size = 4
gram_size = 5

def get_batch(type):
    data = train_data if type=='train' else test_data
    ix randint